In [113]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
import collections
import seaborn as sns
nltk.download('rslp')
from nltk.stem import RSLPStemmer
import heapq as hp
from collections import Counter,OrderedDict
import math

[nltk_data] Downloading package rslp to /home/vinicius/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [114]:
data = pd.read_csv('../lab3/results.csv')

In [134]:
#generate tokens from a document
def parse(doc):
    words = ''.join(str(v) for v in doc).lower()
    return RegexpTokenizer(r'[A-zÀ-ú\d]{4,}').tokenize(words)

# produces: term:[(doc, frequency)]
#build an index for a collection of documents
def build_index_with_frequency(docs):
    index = {}
    n = 0
    for text in docs.texto:
        n +=1
        tokens = parse(text)
        for token in tokens:
            if (token in index):
                hasDoc = False
                for i in range(len(index[token])):
                    tup = index[token][i]
                    if (tup[0] == n):
                        index[token][i] = (n, tup[1] + 1)
                        hasDoc = True
                if (not hasDoc):
                    index[token].append((n,1))
            else:
                index[token] = [(n,1)]
    return index
index = build_index_with_frequency(data)

In [133]:
def document_at_time_retrieval(query, index, n_results):
    inverted_lists = []
    results = []
    
    for term in query.split():
        inverted_lists.append(index[term])
    
    documents = []
    for key in index:
        il = index[key]
        for doc in il:
            documents.append(doc[0])
    documents = list(set(documents))
    
    for doc in documents:
        score = 0
        for il in inverted_lists:
            for d in il:
                if (d[0] == doc):
                    score += d[1]
        results.append((score, doc))
    #[(score, doc)]
    results = sorted(results, reverse=True)
    r = []
    for i in range(n_results):
        if(results[i][0] > 0):
            r.append(results[i])
    return r

In [136]:
def getDocs(iPosition):
    docs = []
    for el in iPosition:
        docs.append(el[0])
    docs = set(docs)
    return docs

def mutual_info(word, index):
    rank = {}
    docs = getDocs(index[word])
    na = len(docs)
    
    for k in index:
        if (k != word):
            db = getDocs(index[k])
            nb = len(db)
            nab = len(docs & db)
            if nab == 0: continue
            rank[k] = nab/(na*nb)
            
    return OrderedDict(sorted(rank.items(), key=lambda x: x[1], reverse=True))

In [130]:
def expected_mutual_info(word,index):
    rank = {}
    n = len(data)
    docs = getDocs(index[word])
    na = len(docs)
    
    for k in index:
        if (k != word):
            db = getDocs(index[k])
            nb = len(db)
            nab = len(docs & db)
            if nab == 0: continue
            rank[k] = nab * math.log(n*(nab/(na*nb)))

            
    return OrderedDict(sorted(rank.items(), key=lambda x: x[1], reverse=True))

In [129]:
def chi_square(word,index):
    rank = {}
    n = len(data)
    docs = getDocs(index[word])
    na = len(docs)
    
    for k in index:
        if (k != word):
            db = getDocs(index[k])
            nb = len(db)
            nab = len(docs & db)
            if nab == 0: continue
            rank[k] = (nab-(1/n)*na*nb)**2 / (na*nb)
     
    return OrderedDict(sorted(rank.items(), key=lambda x: x[1], reverse=True))

In [128]:
def dice(word,index):
    rank = {}
    n = len(data)
    docs = getDocs(index[word])
    na = len(docs)
    
    for k in index:
        if (k != word):
            db = getDocs(index[k])
            nb = len(db)
            nab = len(docs & db)
            if nab == 0: continue
            rank[k] = nab/(na+nb)
     
    return OrderedDict(sorted(rank.items(), key=lambda x: x[1], reverse=True))

## Questão 1:

In [137]:
from IPython.display import display, HTML
queries = ["jogo", "partida", "futebol", "campo", "universidade"]
for query in queries:
    results_mim = []
    for word in mutual_info(query, index): 
        results_mim.append(word)
        if len(results_mim) == 10: break
            
    results_emim = []
    for word in expected_mutual_info(query, index): 
        results_emim.append(word)
        if len(results_emim) == 10: break
    
    results_x2 = []        
    for word in chi_square(query, index): 
        results_x2.append(word)
        if len(results_x2) == 10: break
    
    results_dice = []        
    for word in dice(query, index): 
        results_dice.append(word)
        if len(results_dice) == 10: break
    
    df = pd.DataFrame()
    df['MIM'] = list(results_mim)
    df['EMIM'] = list(results_emim)
    df['X2'] = list(results_x2)
    df['DICE'] = list(results_dice)
    print(query)
    display(df)
    print("\n")

jogo


,MIM,EMIM,X2,DICE
0,atacante,bola,bola,bola
1,profissão,futebol,atacante,futebol
2,sterling,área,oitavas,área
3,yaya,jogador,champions,campo
4,touré,champions,futebol,real
5,kevin,atacante,jogador,jogador
6,prince,oitavas,juve,primeiro
7,boateng,campo,quartas,partida
8,ballotelli,gols,torneio,time
9,lilian,jogadores,atlético,gols




partida


,MIM,EMIM,X2,DICE
0,profissão,transmitida,transmitida,transmitida
1,sterling,horário,horário,horário
2,yaya,jogo,enfrentam,enfrentam
3,touré,enfrentam,esporte,jogo
4,kevin,esporte,torcida,jogador
5,boateng,jogador,juve,confronto
6,ballotelli,confronto,atlético,esporte
7,lilian,ganhar,bater,ganhar
8,thuram,volta,estádio,futebol
9,insulto,estádio,barça,brasília




futebol


,MIM,EMIM,X2,DICE
0,profissão,jogador,jogador,jogador
1,sterling,treinador,treinador,treinador
2,yaya,jogadores,atleta,jogadores
3,touré,seleção,jogadores,atleta
4,kevin,atleta,campeonato,seleção
5,boateng,bola,clubes,bola
6,ballotelli,equipe,torcedores,clubes
7,lilian,jogo,seleção,campeonato
8,thuram,clubes,seleções,time
9,insulto,time,categorias,jogos




campo


,MIM,EMIM,X2,DICE
0,profissão,jogo,chegada,jogo
1,sterling,seleção,articulação,seleção
2,yaya,chegada,seleção,sempre
3,touré,sempre,procedentes,enquanto
4,kevin,enquanto,competição,chegada
5,boateng,futebol,impressionantes,futebol
6,ballotelli,primeiro,completar,primeiro
7,lilian,articulação,residentes,todas
8,thuram,todas,bolívia,importante
9,insulto,importante,pedem,mulheres




universidade


,MIM,EMIM,X2,DICE
0,carolina,professor,antropóloga,professor
1,levantam,muitos,professor,políticas
2,pátria,políticas,pesquisadora,muitos
3,relaxam,exemplo,bairro,exemplo
4,flui,pesquisadora,estudantes,cidades
5,baixam,cidades,pesquisador,século
6,vinham,antropóloga,william,comum
7,bolos,conceito,harvard,esses
8,chocolates,foram,planned,eram
9,ortegano,esses,parenthood,conceito


A partir dos resultados apresentados, acredito que a chi square tenha obtido melhores resultados. 
Para os 4 primeiros termos, o resultado é muito semelhante. No entanto, observando o ultimo, referenque à query `universidade` vemos que a chi square 